In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets
import torch.nn as nn
import torch.optim as optim

from net import Net
from dataset import create_data_loader, split_train_val, getUpperIndices, addASpace

import os
import string

#to do: 
    #shuffle test and train data
    #see if vocab actually gets updated (it seems to be but i dont believe it!)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
#parameters:
numEpochs = 10
batchSize = 1
learningRate = 0.001

paddingValue = 0

#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
print('device:',device)

device: cpu


In [4]:
vocab = ['_PADDING_']


# Turn string into tensor of ints
def tensorConvert(string):
    # runtime can be improved if we find a way to convert tensor using tensor / numpy methods of some sort
    # only thing is i wouldnt know how to do this with a vocab though

    output = []

    tokenize = string.split()
    for token in tokenize:
        if token not in vocab:
            vocab.append(token)

        output.append(vocab.index(token))
        
        #print('\nappended',token,'to vocab!')

    output = torch.LongTensor(output)

    return output

In [5]:
# get list of authors
classes = []
rootdir = 'Data/C50'
for it in os.scandir(rootdir): #scan subdirectory and append each element to list of classes
    if it.is_dir():
        classes.append(it.path.replace(rootdir + "\\" , '')) #remove 'C50train\' from string

print('Num Classes:',len(classes))
print('classes:',classes[0:5])

Num Classes: 50
classes: ['AaronPressman', 'AlanCrosby', 'AlexanderSmith', 'BenjaminKangLim', 'BernardHickey']


In [6]:
#data:
length = sum([len(files) for r, d, files in os.walk("Data/C50")])
print('Number of files:', length)

data = []

def gatherData(path):
    #0 = train, 1 = test
    address = path + "/C50"

        
    prelude = len(address)
    
    for r, d, files in os.walk(address):
        
        #print here to show progress, warning, a shit load of print statements
        
        author = r[(prelude+1):]
        
        
        
        if author == "":
            continue
        
        #print('\nr:',r)
        print('\nAuthor:', author)
        ratio = classes.index(author) / len(classes)
        print(ratio * 100, '% there.')
            
        #EARLY STOP FOR DEVELOPMENT PURPOSES (cuz going through every author takes an ass load time)
        if author == "DarrenSchuettler":
            break

        for file in files:
            address = r + '/' + file

            with open(address, 'r') as f:
                content = f.read()

            convert = tensorConvert(content)
            
            #print('vocab length:',len(vocab))
            #print('sample:',convert[:5])
            
            authorIdx = classes.index(author)
            
            item = [authorIdx, convert]
            #print('item:',item)
            
            data.append(item)
                

Number of files: 5000


In [7]:
gatherData('Data')


Author: AaronPressman
0.0 % there.

Author: AlanCrosby
2.0 % there.

Author: AlexanderSmith
4.0 % there.

Author: BenjaminKangLim
6.0 % there.

Author: BernardHickey
8.0 % there.

Author: BradDorfman
10.0 % there.

Author: DarrenSchuettler
12.0 % there.


In [8]:
print(len(data[0][1]))
print(data[0])

319
[0, tensor([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
         15,  16,  17,  18,   1,  19,  20,  21,  22,  23,  24,  16,  25,  26,
         27,  28,  16,   2,  29,  16,  30,  31,  11,  32,  33,  34,  11,  35,
         36,  37,  38,  39,  40,   6,  41,  15,  42,  43,   1,  44,  35,  45,
         46,  47,  48,  49,  50,  51,  52,  35,  53,  54,  55,  56,  28,  16,
         57,  58,  50,  11,  59,  60,   2,  61,  62,  63,  64,   1,  65,  32,
         66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
         80,  81,  35,  27,  28,  16,   2,  82,  83,  84,  22,  85,  86,  87,
         15,  16,  88,  89,  90,  91,  15,  92,  93,   2,  61,  62,  94,  72,
         35,  95,  96,  97,  16,  98,  11,  99, 100,  51,  11, 101, 102, 103,
        104, 105, 106,  16, 107, 108,   2,  67,  97, 109, 110,  35,  53, 111,
         75, 112, 113,  24, 114, 115,  11, 116, 117, 118, 119,  34, 120, 121,
        122, 123,  35, 124,  69, 125, 126, 127,  15, 128

In [9]:
#find maxLength for padding purposes
maxLength = 0

for item in data:
    tensor = item[1]
    length = len(tensor)
    if length > maxLength:
        maxLength = length
        
print(maxLength)
print(len(data[0][1]))

1082
319


In [10]:
def padTensor(tensor):
    length = len(tensor)

    pad = torch.ones((maxLength - length,))
    pad = pad * paddingValue
    #print('pad:',pad)

    newTensor = torch.cat((tensor, pad),0)
    
    return newTensor
    
for item in data:
    tensor = item[1]
    newTensor = padTensor(tensor)
    
    item[1] = newTensor

item = data[0]
print(item)


[0, tensor([1., 2., 3.,  ..., 0., 0., 0.])]


In [11]:
#Split the dataset
trainData, valData, testData = split_train_val(data, props=[0.8, 0.1, 0.1])

print('Length of data:', len(data))
print('Length of partitions:',len(trainData), len(valData), len(testData))
print('Sum:',len(trainData) + len(valData) + len(testData))



Length of data: 600
Length of partitions: 480 60 60
Sum: 600


In [12]:
trainLoader = create_data_loader(trainData, 
                                      batchSize,
                                      shuffle=True)
valLoader = create_data_loader(testData, 
                                    batchSize,
                                    shuffle=True)

next(iter(trainLoader))

{'input': tensor([[1.0000e+00, 3.1140e+03, 1.5000e+01,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00]]),
 'target': tensor([0])}

In [13]:
#insert model here, need to get numbers for these
#inputSize = len(vocab) #not sure if its supposed to be maxLength or len(vocab)
inputSize = maxLength
hiddenSize = 512
outputSize = len(classes)

net = Net(inputSize, hiddenSize, outputSize).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters(), lr=learningRate)

print('inputSize:',inputSize)

inputSize: 1082


In [14]:
#start training + print accuracy
#i see no reason to split any of this up into multiple cells but feel free to do so if there is one

hidden = net.init_hidden()

for e in range(numEpochs):
    #training loop
    train_loss = 0.0
    net.train()
    for item in trainLoader:
        data = item['input']
        label = item['target']
        
        data = data.long()

        
        #print('data shape:',data.shape)
        #print('data:',data)
        
        data, label = data.to(device), label.to(device)
        
        optimizer.zero_grad()
        output, hidden = net(data, hidden)
        
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    #validation loop
    valid_loss = 0.0
    net.eval()  # Optional when not using Model Specific layer
    for item in validLoader:
        data = item['input']
        label = item['target']
        
        data = data.long()
        
        data, label = data.to(device), label.to(device)

        output = net(data)
        loss = criterion(output, label)
        valid_loss = loss.item() * data.size(0)

    print(
        f'Epoch {e + 1} \t\t Training Loss: {train_loss / len(trainLoader)} \t\t Validation Loss: {valid_loss / len(validLoader)}')

    #save model if validation loss decreases
    if min_valid_loss > valid_loss:
        print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
        min_valid_loss = valid_loss
        # Saving State Dict
        torch.save(net.state_dict(), 'Models/saved_model.pth')

#test loop
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

with torch.no_grad():
    for data in testLoader:
        data = item['input']
        label = item['target']
        
        data = data.long()
        
        outputs = net(data)
        _, predictions = torch.max(outputs, 1)

        if label == prediction:
            correct_pred[classes[label]] += 1
        total_pred[classes[label]] += 1

for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname, accuracy))

test: torch.Size([1, 1082])
test: tensor([[   1, 3114,   15,  ...,    0,    0,    0]])


IndexError: index out of range in self